In [1]:
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import torch
import torch
from sklearn.model_selection import StratifiedKFold


In [2]:
df=pd.read_csv('cleaned.csv',index_col=0)
testset=pd.read_csv('test.csv',index_col=0)
X = df.drop('SMOKING', axis=1)
y = df['SMOKING']

X_test_new = testset.drop('SMOKING', axis=1)
y_test_new = testset['SMOKING']

In [3]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score
import numpy as np

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

def objective(trial):
    
    n_d = trial.suggest_int("n_d", 8, 32, step=4)       
    n_a = trial.suggest_int("n_a", 8, 32, step=4)       
    n_steps = trial.suggest_int("n_steps", 3, 6)        
    gamma = trial.suggest_float("gamma", 1.0, 1.5)      
    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-5, 1e-3, log=True)
    lr = trial.suggest_float("lr", 1e-3, 5e-2, log=True)  
    max_epochs = trial.suggest_int("max_epochs", 20, 50)  

    scores = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_fold)
        X_val_scaled   = scaler.transform(X_val_fold)

        #  numpy
        X_train_fold = X_train_scaled.astype(np.float32)
        X_val_fold   = X_val_scaled.astype(np.float32)
        y_train_fold = y_train_fold.values.astype(np.int64)
        y_val_fold   = y_val_fold.values.astype(np.int64)

        model = TabNetClassifier(
            n_d=n_d,
            n_a=n_a,
            n_steps=n_steps,
            gamma=gamma,
            lambda_sparse=lambda_sparse,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=lr),
            verbose=50,
            seed=42
        )

        model.fit(
            X_train_fold, y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric=['accuracy'],
            max_epochs=max_epochs,
            patience=5,
            batch_size=32,
            virtual_batch_size=32
        )

        y_pred_val = model.predict(X_val_fold)
        acc = accuracy_score(y_val_fold, y_pred_val)
        
        scores.append(acc)


    return np.mean(scores)


sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=20)

print("best_params:", study.best_params)
print("best_value:", study.best_value)

[I 2025-08-16 09:42:04,761] A new study created in memory with name: no-name-1fb8f454-5816-43a8-8c8c-38f9479ebf15
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.63527 | val_0_accuracy: 0.54    |  0:00:00s

Early stopping occurred at epoch 9 with best_epoch = 4 and best_val_0_accuracy = 0.7
epoch 0  | loss: 1.61433 | val_0_accuracy: 0.6     |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")



Early stopping occurred at epoch 5 with best_epoch = 0 and best_val_0_accuracy = 0.6
epoch 0  | loss: 1.45298 | val_0_accuracy: 0.53061 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...


Early stopping occurred at epoch 16 with best_epoch = 11 and best_val_0_accuracy = 0.73469
epoch 0  | loss: 0.99676 | val_0_accuracy: 0.44898 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")



Early stopping occurred at epoch 6 with best_epoch = 1 and best_val_0_accuracy = 0.61224
epoch 0  | loss: 0.98874 | val_0_accuracy: 0.65306 |  0:00:00s

Early stopping occurred at epoch 5 with best_epoch = 0 and best_val_0_accuracy = 0.65306


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[I 2025-08-16 09:42:48,908] Trial 19 finished with value: 0.696 and parameters: {'n_d': 32, 'n_a': 24, 'n_steps': 3, 'gamma': 1.2715536483939176, 'lambda_sparse': 0.00011982448134112166, 'lr': 0.012364555319037338, 'max_epochs': 43}. Best is trial 11 with value: 0.7815510204081633.


best_params: {'n_d': 32, 'n_a': 24, 'n_steps': 4, 'gamma': 1.3712182258436791, 'lambda_sparse': 0.00019905888149829976, 'lr': 0.024971811264939553, 'max_epochs': 24}
best_value: 0.7815510204081633


In [4]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]   

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_fold)
    X_val_scaled   = scaler.transform(X_val_fold)

  
    X_train_fold = X_train_scaled.astype(np.float32)
    X_val_fold   = X_val_scaled.astype(np.float32)
    y_train_fold = y_train_fold.values.astype(np.int64)
    y_val_fold   = y_val_fold.values.astype(np.int64)

    model = TabNetClassifier(
        n_d=32,
        n_a=24,
        n_steps=4,
        gamma=1.3712182258436791,
        lambda_sparse=0.00019905888149829976,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=0.024971811264939553),
        verbose=50,
        seed=42
    )
    #best_params: {'n_d': 32, 'n_a': 24, 'n_steps': 4, 'gamma': 1.3712182258436791, 'lambda_sparse': 0.00019905888149829976, 'lr': 0.024971811264939553, 'max_epochs': 24}


    model.fit(
        X_train_fold, y_train_fold,
        eval_set=[(X_val_fold, y_val_fold)],
        eval_metric=['accuracy'],
        max_epochs=24,
        patience=5,
        batch_size=32,
        virtual_batch_size=32
    )

    y_pred_val = model.predict(X_val_fold)
    acc = accuracy_score(y_val_fold, y_pred_val)


    y_train_pred = model.predict(X_train_fold)

    train_acc = accuracy_score(y_train_fold, y_train_pred)
    print(f"TabNet Train Accuracy (full-train): {train_acc:.4f}")

    y_test_pred = model.predict(X_val_fold)
    test_acc = accuracy_score(y_val_fold, y_test_pred)
    print(f"\nTabNet Test Accuracy: {test_acc:.4f}")
    print("Test Classification Report:\n", classification_report(y_val_fold, y_test_pred, digits=4))

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.41217 | val_0_accuracy: 0.54    |  0:00:00s

Early stopping occurred at epoch 11 with best_epoch = 6 and best_val_0_accuracy = 0.74
TabNet Train Accuracy (full-train): 0.7005

TabNet Test Accuracy: 0.7400
Test Classification Report:
               precision    recall  f1-score   support

           0     0.8000    0.5455    0.6486        22
           1     0.7143    0.8929    0.7937        28

    accuracy                         0.7400        50
   macro avg     0.7571    0.7192    0.7211        50
weighted avg     0.7520    0.7400    0.7298        50

epoch 0  | loss: 1.1493  | val_0_accuracy: 0.5     |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")



Early stopping occurred at epoch 13 with best_epoch = 8 and best_val_0_accuracy = 0.78
TabNet Train Accuracy (full-train): 0.7411

TabNet Test Accuracy: 0.7800
Test Classification Report:
               precision    recall  f1-score   support

           0     0.8667    0.5909    0.7027        22
           1     0.7429    0.9286    0.8254        28

    accuracy                         0.7800        50
   macro avg     0.8048    0.7597    0.7640        50
weighted avg     0.7973    0.7800    0.7714        50

epoch 0  | loss: 0.99131 | val_0_accuracy: 0.46939 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


...

/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")



Early stopping occurred at epoch 7 with best_epoch = 2 and best_val_0_accuracy = 0.79592
TabNet Train Accuracy (full-train): 0.6313

TabNet Test Accuracy: 0.7959
Test Classification Report:
               precision    recall  f1-score   support

           0     0.8235    0.6667    0.7368        21
           1     0.7812    0.8929    0.8333        28

    accuracy                         0.7959        49
   macro avg     0.8024    0.7798    0.7851        49
weighted avg     0.7994    0.7959    0.7920        49

epoch 0  | loss: 1.31789 | val_0_accuracy: 0.53061 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")



Early stopping occurred at epoch 20 with best_epoch = 15 and best_val_0_accuracy = 0.81633
TabNet Train Accuracy (full-train): 0.8384

TabNet Test Accuracy: 0.8163
Test Classification Report:
               precision    recall  f1-score   support

           0     0.8095    0.7727    0.7907        22
           1     0.8214    0.8519    0.8364        27

    accuracy                         0.8163        49
   macro avg     0.8155    0.8123    0.8135        49
weighted avg     0.8161    0.8163    0.8159        49



/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X)
X_val_scaled   = scaler.transform(X_test_new)


X_train_fold = X_train_scaled.astype(np.float32)
X_val_fold   = X_val_scaled.astype(np.float32)
y_train_fold = y.values.astype(np.int64)
y_val_fold   = y_test_new.values.astype(np.int64)

model = TabNetClassifier(
    n_d=32,
        n_a=24,
        n_steps=4,
        gamma=1.3712182258436791,
        lambda_sparse=0.00019905888149829976,
        optimizer_fn=torch.optim.Adam,
        optimizer_params=dict(lr=0.024971811264939553),
        verbose=50,
        seed=42
)

model.fit(
    X_train_fold, y_train_fold,
        # eval_set=[(X_val_fold, y_val_fold)],
        # eval_metric=['accuracy'],
        # patience=5,
        max_epochs=24,
        batch_size=32,
        virtual_batch_size=32
)

y_pred_val = model.predict(X_val_fold)
acc = accuracy_score(y_val_fold, y_pred_val)


y_train_pred = model.predict(X_train_fold)

train_acc = accuracy_score(y_train_fold, y_train_pred)
print(f"TabNet Train Accuracy (full-train): {train_acc:.4f}")

y_test_pred = model.predict(X_val_fold)
test_acc = accuracy_score(y_val_fold, y_test_pred)
print(f"\nTabNet Test Accuracy: {test_acc:.4f}")
print("Test Classification Report:\n", classification_report(y_val_fold, y_test_pred, digits=4))


epoch 0  | loss: 1.10446 |  0:00:00s


/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
/opt/anaconda3/lib/python3.12/site-packages/pytorch_tabnet/abstract_model.py:687: UserWarning: No early stopping will be performed, last training weights will be used.
  warnings.warn(wrn_msg)


TabNet Train Accuracy (full-train): 0.8421

TabNet Test Accuracy: 0.7581
Test Classification Report:
               precision    recall  f1-score   support

           0     0.7143    0.7407    0.7273        27
           1     0.7941    0.7714    0.7826        35

    accuracy                         0.7581        62
   macro avg     0.7542    0.7561    0.7549        62
weighted avg     0.7594    0.7581    0.7585        62



In [6]:
y_score_tab = model.predict_proba(X_val_fold)[:, 1]
df_scores = pd.read_csv("roc_scores_rf.csv")   
df_scores["tab"] = y_score_tab
df_scores.to_csv("roc_scores_rf.csv", index=False)